# Package Installation and Data Reading

In [2]:
import numpy as np
import pandas as pd

In [3]:
# go back one folder and read the file (relative path)
pd.read_csv("/Users/alexholzer/Desktop/retail_price_prediction/data/file_out.csv")

,Unnamed: 0,DocumentID,Date,SKU,Price,Discount,Customer,Quantity
0,0,716,2019-09-23,1039,381.780000,67.372540,1,1.0
1,1,716,2019-09-23,853,593.220000,0.000340,1,1.0
2,2,716,2019-09-23,862,423.730000,-0.001190,1,1.0
3,3,716,2019-09-23,868,201.700000,35.588140,1,1.0
4,4,716,2019-09-23,2313,345.760000,61.019660,1,1.0
...,...,...,...,...,...,...,...,...
33351,33351,13315,2022-11-09,1849,5957.627119,1072.372881,239,4.0
33352,33352,13316,2022-11-09,30,1271.186441,228.813559,477,2.0
33353,33353,13317,2022-11-09,2066,1355.932203,244.067797,370,1.0
33354,33354,13318,2022-11-09,1586,2334.745763,420.254237,356,2.0


In [5]:
pd.read_csv("/Users/alexholzer/Desktop/retail_price_prediction/data/file_out2.csv")

,Unnamed: 0,InvoiceID,Date,ProductID,TotalSales,Discount,CustomerID,Quantity
0,0,328,2019-12-27,1684,796.610169,143.389831,185,4
1,1,329,2019-12-27,524,355.932203,64.067797,185,2
2,2,330,2019-12-27,192,901.694915,162.305085,230,4
3,3,330,2019-12-27,218,182.754237,32.895763,230,1
4,4,330,2019-12-27,247,780.101695,140.418305,230,4
...,...,...,...,...,...,...,...,...
29098,29098,11092,2023-01-13,1644,6573.000000,1183.140000,269,1
29099,29099,11093,2023-01-13,352,5179.728814,932.351186,250,4
29100,29100,11094,2023-01-13,683,7741.423729,1393.456271,415,4
29101,29101,11095,2023-01-14,1830,3644.067797,655.932203,59,4
